In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import time    

In [3]:
transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])

#train_data_path = 'relative path of training data set'
train_data_path = './horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=500, shuffle=False, num_workers=1)  

validation_data_path = './horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=500, shuffle=False, num_workers=1)  


trainList = list()
validList = list()
trainLabelList = list()
validLabelList = list()

for i, data in enumerate(trainloader):
    # inputs is the image
    # labels is the class of the image
    inputs, labels = data


    # if you don't change the image size, it will be [batch_size, 1, 100, 100]

    batch_size = inputs.shape[0]

    # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
    # change inputs to matrix 10000*batch_size
    for bat_idx in range(batch_size):

        targMat = inputs[bat_idx][0]

        colVec = np.reshape(targMat, (np.product(targMat.shape), 1), 'F')

        if(bat_idx == 0):
            batMat = colVec
        else:
            batMat = np.concatenate((batMat, colVec), axis = 1)         

    # Add ones because of the value b in coefficient
    ones = np.ones((1, batch_size), dtype = int)
    batMat = np.concatenate((batMat, ones))
    trainList.append(batMat)
    trainLabelList.append(labels)
    
# load validation images of the batch size for every iteration
for i, data in enumerate(valloader):

    # inputs is the image
    # labels is the class of the image
    inputs, labels = data

    # if you don't change the image size, it will be [batch_size, 1, 100, 100]
     # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]

    batch_size = inputs.shape[0]

    # Change Inputs to matrix 10000*batch_size

    for bat_idx in range(batch_size):
        targMat = inputs[bat_idx][0]
        colVec = np.reshape(targMat, (np.product(targMat.shape), 1), 'F')

        if(bat_idx == 0):
            batMat = colVec
        else:
            batMat = np.concatenate((batMat,colVec), axis = 1)
        
    # Add ones because of the value b in coefficient
    ones = np.ones((1, batch_size), dtype = int)
    batMat = np.concatenate((batMat, ones))
    validList.append(batMat)
    validLabelList.append(labels)

In [4]:
#Initialize Coef to Zeros
w_0 = np.zeros((4,10001), dtype = float)
w_1 = np.zeros((3, 4), dtype = float)
w_2 = np.zeros((1, 3), dtype = float)

#Set Learning Rate
lrnRate = 0.002

# Set Loss Lists
lrnLoss = list()
valLoss = list()

# Set Accurate Lists
lrnAcc = list()
valAcc = list()

# set Elapsed time Lists
elapTime = list()


epoch = -1
lrnAccRate = 0

while(lrnAccRate < 0.85):
    epoch += 1
    # load training images of the batch size for every iteration
    
    #Set Sum of Derivatives to 0
    sumDw_0 = np.zeros((10001,4), dtype = float)
    
    #Set Sum of Loss to 0
    sumL = 0
    
    #Set Sum of Cor to 0
    cor = 0
    
    start_time = time.time()
    for i, a_0 in enumerate(trainList):
        
        batch_size = a_0.shape[1]


        # Start Regression Calculation
        z_0 = np.dot(w_0, a_0)                                           #coef.shape = (1,10001), batMat.shape = (10001, batch_size), z.shape = (1, batch_size)
        a_1 = 1/(1 + np.exp(-z_0))                                                 #a.shape = (1, batch_size)
        
        z_1 = np.dot(w_1, a_1)
        a_2 = 1/(1 + np.exp(-z_1))
        
        z_2 = np.dot(w_2, a_2)
        a_3 = 1/(1 + np.exp(-z_2))
        
        
        # Calculate Total Loss
        a_3 = torch.from_numpy(a_3)                       #change ndarray to tensor
        dLabels = trainLabelList[i].double()                      #change tensor type to double
        L = -(dLabels) * np.log(a_3) - (1-dLabels) * np.log(1-a_3)      #labels.shape = (1, batch_size), L.shape = (1,batch_size)
        sumL += L.sum()
        
        
        
        # Calculate Accuracy
        

        for batIdx in range(batch_size):

            if(a_3[0][batIdx] <= 0.5 and trainLabelList[i][batIdx] == 0):
                cor += 1

            if(a_3[0][batIdx] > 0.5 and trainLabelList[i][batIdx] == 1):
                cor += 1
            
        
    
    totalDataNum = len(trainloader.dataset)
        
    # Calculate dLossdCoef

    

    # Update coefs using derivatives
    
    
    elapsed_time = time.time() - start_time
    elapTime.append(elapsed_time)


    # Calculate TotalLoss
    sumL /= totalDataNum

    lrnLoss.append(sumL)
    
    
    # Calculate Accuracy
    
    lrnAccRate = cor/totalDataNum
    lrnAcc.append(lrnAccRate)
    
                


    # Set Sum Of Valid Loss to 0
    sumVL = 0
    # Set Sum of Valid Cor to 0
    vCor = 0
    
    # load validation images of the batch size for every iteration
    for i, a_0 in enumerate(validList):
        
        
        batch_size = batMat.shape[1]
        
        # Start Calculate Loss 
        z_0 = np.dot(w_0, a_0)
        a_1= 1/(1+np.exp(-z_0))
        
        z_1 = np.dot(w_1, a_1)
        a_2= 1/(1+np.exp(-z_1))
        
        z_2 = np.dot(w_2, a_2)
        a_3= 1/(1+np.exp(-z_2))
        
        
        a = torch.from_numpy(a)
        dLabels = validLabelList[i].double()
        L = -(dLabels) * np.log(a) - (1-dLabels) * np.log(1-a)
        sumVL += L.sum()

        
        # Calculate Accuracy
        
        for batIdx in range(batch_size):
            if(a_3[0][batIdx] <= 0.5 and validLabelList[i][batIdx] == 0):
                vCor += 1
                
            if(a_3[0][batIdx] > 0.5 and validLabelList[i][batIdx] == 1):
                vCor += 1
    
    totalValDataNum = len(valloader.dataset)
    
    # CalCulate Total Loss
    sumVL /= totalValDataNum
    valLoss.append(sumVL)
    
    
    # Calculate Accuracy
    vAcc = vCor/totalValDataNum
    valAcc.append(vAcc)
    
    
    print("epoch : ",epoch,',', "lrnCor : ", cor, "valCor : ", vCor, "elapsed time : ", elapsed_time)
    
    

RuntimeError: The size of tensor a (500) must match the size of tensor b (256) at non-singleton dimension 1

In [10]:
#Initialize Coef to Zeros
coef = np.zeros((1,10001), dtype = float)

#Set Learning Rate
lrnRate = 0.002

# Set Loss Lists
lrnLoss = list()
valLoss = list()

# Set Accurate Lists
lrnAcc = list()
valAcc = list()

# set Elapsed time Lists
elapTime = list()


epoch = -1
lrnAccRate = 0
while(lrnAccRate < 0.85):
    epoch += 1
    # load training images of the batch size for every iteration
    
    #Set Sum of Derivatives to 0
    sumDCoef = np.zeros((10001,1), dtype = float)
    
    #Set Sum of Loss to 0
    sumL = 0
    
    #Set Sum of Cor to 0
    cor = 0
    
    start_time = time.time()
    for i, batMat in enumerate(trainList):
        
        batch_size = batMat.shape[1]


        # Start Regression Calculation
        z = np.dot(coef, batMat)                                           #coef.shape = (1,10001), batMat.shape = (10001, batch_size), z.shape = (1, batch_size)
        a = 1/(1 + np.exp(-z))                                                 #a.shape = (1, batch_size)
        dz = np.subtract(a, trainLabelList[i])                                        #dz.shape = (1, batch_size)
        batMat = torch.from_numpy(batMat)                       #change ndarray to tensor
        dCoef = dz * batMat                                                 #dCoef.shape = (10001, batch_size)
        sumHelper = np.ones((batch_size,1), dtype = int)    #sumHelper.shape = (batch_size,1)
        sumDCoef += np.dot(dCoef, sumHelper)                 #sumDCoef.shape = (10001, 1)
        
        
        # Calculate Total Loss
        a = torch.from_numpy(a)                       #change ndarray to tensor
        dLabels = trainLabelList[i].double()                      #change tensor type to double
        L = -(dLabels) * np.log(a) - (1-dLabels) * np.log(1-a)      #labels.shape = (1, batch_size), L.shape = (1,batch_size)
        sumL += L.sum()
        
        
        
        # Calculate Accuracy
        

        for batIdx in range(batch_size):

            if(a[0][batIdx] <= 0.5 and trainLabelList[i][batIdx] == 0):
                cor += 1

            if(a[0][batIdx] > 0.5 and trainLabelList[i][batIdx] == 1):
                cor += 1
            
        
    
    totalDataNum = len(trainloader.dataset)
        
    # Calculate dLossdCoef

    sumDCoef /= totalDataNum

    # Update coefs using derivatives
    coef = coef.T
    coef -= lrnRate * sumDCoef
    coef = coef.T
    
    elapsed_time = time.time() - start_time
    elapTime.append(elapsed_time)


    # Calculate TotalLoss
    sumL /= totalDataNum

    lrnLoss.append(sumL)
    
    
    # Calculate Accuracy
    
    lrnAccRate = cor/totalDataNum
    lrnAcc.append(lrnAccRate)
    
                


    # Set Sum Of Valid Loss to 0
    sumVL = 0
    # Set Sum of Valid Cor to 0
    vCor = 0
    
    # load validation images of the batch size for every iteration
    for i, batMat in enumerate(validList):
        
        
        batch_size = batMat.shape[1]
        
        # Start Calculate Loss 
        z = np.dot(coef, batMat)
        a = 1/(1+np.exp(-z))
        a = torch.from_numpy(a)
        dLabels = validLabelList[i].double()
        L = -(dLabels) * np.log(a) - (1-dLabels) * np.log(1-a)
        sumVL += L.sum()

        
        # Calculate Accuracy
        
        for batIdx in range(batch_size):
            if(a[0][batIdx] <= 0.5 and validLabelList[i][batIdx] == 0):
                vCor += 1
                
            if(a[0][batIdx] > 0.5 and validLabelList[i][batIdx] == 1):
                vCor += 1
    
    totalValDataNum = len(valloader.dataset)
    
    # CalCulate Total Loss
    sumVL /= totalValDataNum
    valLoss.append(sumVL)
    
    
    # Calculate Accuracy
    vAcc = vCor/totalValDataNum
    valAcc.append(vAcc)
    
    
    print("epoch : ",epoch,',', "lrnCor : ", cor, "valCor : ", vCor, "elapsed time : ", elapsed_time)
    
    

[[3.24799047e-05 2.88752691e-05 2.54997418e-05 ... 4.29921541e-05
  4.29730619e-05 2.62901655e-05]]
[[-1.32287528e-05 -2.04482926e-05 -2.72442512e-05 ...  1.77635305e-06
   1.92533524e-06 -7.66217493e-05]]
[[-1.84974887e-05 -2.92964863e-05 -3.94748585e-05 ...  2.66371323e-06
   2.87961260e-06 -1.13604805e-04]]
[[-4.28207737e-05 -5.71882420e-05 -7.07528944e-05 ... -1.76219832e-05
  -1.72954024e-05 -1.82495249e-04]]
[[-5.65264048e-05 -7.44396975e-05 -9.13733880e-05 ... -2.74697001e-05
  -2.70607908e-05 -2.34474059e-04]]
[[-7.45085137e-05 -9.59518022e-05 -1.16248123e-04 ... -4.26906820e-05
  -4.21912318e-05 -2.94010802e-04]]
[[-8.93666123e-05 -1.14321362e-04 -1.37968590e-04 ... -5.53782110e-05
  -5.47978921e-05 -3.48913982e-04]]
[[-1.04824965e-04 -1.33274794e-04 -1.60264054e-04 ... -6.94608360e-05
  -6.88000023e-05 -4.05285400e-04]]
[[-1.19064076e-04 -1.50992252e-04 -1.81313402e-04 ... -8.29716547e-05
  -8.22349325e-05 -4.60112231e-04]]
[[-1.33018626e-04 -1.68409326e-04 -2.02053030e-04 ..

[[-0.00033072 -0.00064271 -0.00094416 ... -0.00120662 -0.00119867
  -0.00422227]]
[[-0.00032832 -0.00064325 -0.00094747 ... -0.00121663 -0.00120855
  -0.0042566 ]]
[[-0.00032588 -0.00064375 -0.00095074 ... -0.00122659 -0.00121838
  -0.00429083]]
[[-0.00032341 -0.00064422 -0.00095397 ... -0.0012365  -0.00122815
  -0.00432496]]
[[-0.00032091 -0.00064465 -0.00095716 ... -0.00124637 -0.00123788
  -0.004359  ]]
[[-0.00031838 -0.00064504 -0.00096032 ... -0.00125618 -0.00124755
  -0.00439295]]
[[-0.00031582 -0.00064541 -0.00096343 ... -0.00126594 -0.00125718
  -0.0044268 ]]
[[-0.00031322 -0.00064574 -0.00096652 ... -0.00127566 -0.00126676
  -0.00446056]]
[[-0.0003106  -0.00064604 -0.00096956 ... -0.00128533 -0.00127628
  -0.00449423]]
[[-0.00030795 -0.00064631 -0.00097257 ... -0.00129495 -0.00128576
  -0.00452781]]
[[-0.00030527 -0.00064655 -0.00097555 ... -0.00130452 -0.00129519
  -0.0045613 ]]
[[-0.00030256 -0.00064676 -0.0009785  ... -0.00131405 -0.00130457
  -0.0045947 ]]
[[-0.00029983 -0

[[-2.30279706e-05 -6.07409827e-04 -1.15697490e-03 ... -1.98237183e-03
  -1.95725348e-03 -7.16030121e-03]]
[[-1.94045563e-05 -6.06595565e-04 -1.15863042e-03 ... -1.98910379e-03
  -1.96376667e-03 -7.18862629e-03]]
[[-1.57787936e-05 -6.05777889e-04 -1.16027997e-03 ... -1.99581350e-03
  -1.97025703e-03 -7.21691051e-03]]
[[-1.21507778e-05 -6.04956894e-04 -1.16192364e-03 ... -2.00250114e-03
  -1.97672474e-03 -7.24515417e-03]]
[[-8.52060271e-06 -6.04132678e-04 -1.16356155e-03 ... -2.00916689e-03
  -1.98317000e-03 -7.27335757e-03]]
[[-4.88836015e-06 -6.03305333e-04 -1.16519379e-03 ... -2.01581094e-03
  -1.98959298e-03 -7.30152101e-03]]
[[-1.25414025e-06 -6.02474953e-04 -1.16682047e-03 ... -2.02243344e-03
  -1.99599387e-03 -7.32964477e-03]]
[[ 2.38196858e-06 -6.01641628e-04 -1.16844169e-03 ... -2.02903459e-03
  -2.00237284e-03 -7.35772914e-03]]
[[ 6.01987958e-06 -6.00805445e-04 -1.17005754e-03 ... -2.03561454e-03
  -2.00873007e-03 -7.38577442e-03]]
[[ 9.65950762e-06 -5.99966493e-04 -1.17166813e

[[ 0.00030822 -0.0005276  -0.00129242 ... -0.00252164 -0.00247479
  -0.00960037]]
[[ 0.00031182 -0.00052672 -0.00129381 ... -0.00252691 -0.0024798
  -0.00962591]]
[[ 0.00031541 -0.00052585 -0.0012952  ... -0.00253216 -0.0024848
  -0.00965144]]
[[ 0.000319   -0.00052497 -0.00129659 ... -0.0025374  -0.00248979
  -0.00967694]]
[[ 0.00032259 -0.0005241  -0.00129798 ... -0.00254264 -0.00249477
  -0.00970242]]
[[ 0.00032617 -0.00052323 -0.00129937 ... -0.00254786 -0.00249973
  -0.00972788]]
[[ 0.00032975 -0.00052236 -0.00130076 ... -0.00255307 -0.00250469
  -0.00975332]]
[[ 0.00033334 -0.00052149 -0.00130215 ... -0.00255826 -0.00250963
  -0.00977874]]
[[ 0.00033692 -0.00052062 -0.00130353 ... -0.00256345 -0.00251456
  -0.00980413]]
[[ 0.00034049 -0.00051975 -0.00130492 ... -0.00256863 -0.00251948
  -0.0098295 ]]
[[ 0.00034407 -0.00051888 -0.0013063  ... -0.00257379 -0.00252439
  -0.00985485]]
[[ 0.00034764 -0.00051802 -0.00130769 ... -0.00257895 -0.00252929
  -0.00988018]]
[[ 0.00035121 -0.0

[[ 0.00065909 -0.00044637 -0.00143142 ... -0.0030057  -0.0029324
  -0.01208569]]
[[ 0.00066243 -0.00044566 -0.0014328  ... -0.0030101  -0.00293653
  -0.0121095 ]]
[[ 0.00066576 -0.00044495 -0.00143419 ... -0.00301449 -0.00294065
  -0.01213329]]
[[ 0.00066909 -0.00044423 -0.00143557 ... -0.00301887 -0.00294477
  -0.01215707]]
[[ 0.00067242 -0.00044353 -0.00143695 ... -0.00302324 -0.00294887
  -0.01218084]]
[[ 0.00067575 -0.00044282 -0.00143834 ... -0.00302761 -0.00295298
  -0.01220459]]
[[ 0.00067907 -0.00044211 -0.00143972 ... -0.00303197 -0.00295707
  -0.01222833]]
[[ 0.00068239 -0.00044141 -0.00144111 ... -0.00303633 -0.00296116
  -0.01225206]]
[[ 0.00068571 -0.00044071 -0.00144249 ... -0.00304068 -0.00296524
  -0.01227578]]
[[ 0.00068902 -0.00044001 -0.00144388 ... -0.00304502 -0.00296932
  -0.01229948]]
[[ 0.00069233 -0.00043931 -0.00144527 ... -0.00304936 -0.00297339
  -0.01232317]]
[[ 0.00069564 -0.00043861 -0.00144665 ... -0.00305369 -0.00297745
  -0.01234685]]
[[ 0.00069895 -0.

[[ 0.00097863 -0.00038465 -0.00157252 ... -0.00341751 -0.00331749
  -0.01441029]]
[[ 0.00098169 -0.00038413 -0.00157396 ... -0.00342139 -0.00332109
  -0.01443305]]
[[ 0.00098473 -0.00038362 -0.00157541 ... -0.00342526 -0.0033247
  -0.01445579]]
[[ 0.00098778 -0.00038311 -0.00157686 ... -0.00342913 -0.0033283
  -0.01447853]]
[[ 0.00099082 -0.00038259 -0.0015783  ... -0.00343299 -0.0033319
  -0.01450125]]
[[ 0.00099386 -0.00038209 -0.00157975 ... -0.00343685 -0.00333549
  -0.01452397]]
[[ 0.0009969  -0.00038158 -0.0015812  ... -0.0034407  -0.00333908
  -0.01454668]]
[[ 0.00099994 -0.00038107 -0.00158265 ... -0.00344455 -0.00334266
  -0.01456939]]
[[ 0.00100297 -0.00038057 -0.0015841  ... -0.0034484  -0.00334624
  -0.01459208]]
[[ 0.001006   -0.00038007 -0.00158555 ... -0.00345224 -0.00334982
  -0.01461477]]
[[ 0.00100903 -0.00037957 -0.001587   ... -0.00345608 -0.00335339
  -0.01463745]]
[[ 0.00101205 -0.00037907 -0.00158845 ... -0.00345991 -0.00335696
  -0.01466012]]
[[ 0.00101507 -0.00

In [ ]:
plt.plot(elapTime, color = 'black', label = "ElapsedTime")
plt.xlabel("Iteration")
plt.ylabel("time")
plt.legend()



In [ ]:
#Plotting Loss

fig, axs = plt.subplots(2,1, figsize = (10,4))


axs[0].plot(lrnLoss, color = 'red', label = "TrainingLoss")


axs[1].plot(valLoss, color = 'blue', label = "ValidationLoss")

axs[0].set(ylabel = 'Loss')
axs[1].set(xlabel = 'Iteration', ylabel = 'Loss')

axs[0].legend()
axs[1].legend()

In [ ]:
#Plotting Accuracy
fig, axs = plt.subplots(2,1, figsize = (10,5))


axs[0].plot(lrnAcc, color = 'orange', label = "TrainingAccuracy")


axs[1].plot(valAcc, color = 'green', label = "ValidationAccuracy")

axs[0].set(ylabel = 'Accuracy')
axs[1].set(xlabel = 'Iteration', ylabel = 'Accuracy')

axs[0].legend()
axs[1].legend()

In [ ]:
#Plotting Altogether

plt.figure(figsize = (20,10))
plt.title('Altogether')
plt.xlabel('Iteration')

plt.plot(lrnLoss, color = 'red', label = 'TrainingLoss')
plt.plot(valLoss, color = 'blue', label = "ValidationLoss")
plt.plot(lrnAcc, color = 'orange', label = "TrainingAccuracy")
plt.plot(valAcc, color = 'green', label = "ValidationAccuracy")

plt.legend()






In [ ]:
print("  Dataset   |   Loss   | Accuracy")

print("  Training  | %.4f | %.4f" % (lrnLoss[-1], lrnAcc[-1]))
print(" Validation| %.4f | %.4f" % (valLoss[-1], valAcc[-1]))